# **Reproducing M1 results (Table 2)**

<a target="_blank" href="https://colab.research.google.com/github/blackswan-advitamaeternam/HVAE/blob/raph/paper_experiments/Table2_exp.ipynb"> <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/> </a>

## **Colab setup**

In [1]:
from google.colab import drive
drive.mount('/content/drive')
# to avoid having the data on your drive
%cd /content

Mounted at /content/drive
/content


In [2]:
!git clone https://github.com/blackswan-advitamaeternam/HVAE.git
%cd HVAE
!git checkout raph
!pip install -r requirements.txt

Cloning into 'HVAE'...
remote: Enumerating objects: 362, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 362 (delta 10), reused 7 (delta 3), pack-reused 317 (from 1)
Receiving objects: 100% (362/362), 1.47 MiB | 4.80 MiB/s, done.
Resolving deltas: 100% (232/232), done.
/content/HVAE
Branch 'raph' set up to track remote branch 'raph' from 'origin'.
Switched to a new branch 'raph'
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 64.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.2/174.2 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.6/58.6 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.1/284.1 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 147.6 MB/s eta 0:00:00


To allow automatic reloading

In [3]:
!pip install --upgrade ipython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 621.4/621.4 kB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 89.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 9.0 MB/s eta 0:00:00
  Attempting uninstall: traitlets
    Found existing installation: traitlets 5.7.1
    Uninstalling traitlets-5.7.1:
      Successfully uninstalled traitlets-5.7.1
  Attempting uninstall: ipython
    Found existing installation: ipython 7.34.0
    Uninstalling ipython-7.34.0:
      Successfully uninstalled ipython-7.34.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires ipython==7.34.0, but you have ipython 9.8.0 which is incompatible.


In [4]:
import sys
try:
    import imp
except ImportError:
    import types
    sys.modules['imp'] = types.ModuleType('imp')

In [5]:
%load_ext autoreload
%autoreload 2

## **Imports**

In [6]:
import sys, os
# To ensure the custom package is found
path_to_repo = "/content/HVAE"
if path_to_repo not in sys.path:
    sys.path.append(path_to_repo)

In [7]:
import numpy as np
import pandas as pd
import torch
from tqdm.auto import tqdm
from sklearn.metrics import accuracy_score

from svae.vae import M1, predict_classes_loader
from svae.training import training_M1
from svae.utils import ShuffledLoader

from paper_experiments.load_MNIST import make_splits_loaders_MNIST

/content/HVAE/paper_experiments/load_MNIST.py
/content/HVAE/paper_experiments


Setting device

In [8]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [9]:
# make splits
NUM_WORKERS = int(0.8*os.cpu_count())
FRAC = 0.1
TRAIN_FRAC = int(50000 * FRAC)
VAL_FRAC = int(10000 * FRAC)
TEST_FRAC = None # potentially still test on all test samples
print(f"Using {NUM_WORKERS} workers.")
train_loader, val_loader, test_loader = make_splits_loaders_MNIST(train_size=TRAIN_FRAC, val_size=VAL_FRAC, test_size=TEST_FRAC,
                                                                batch_size=64,
                                                                test_batch_size=100,
                                                                num_workers=NUM_WORKERS,
                                                                prefetch_factor=2,
                                                                force=True,
                                                                persistent_workers=True,
                                                                pin_memory=True)

Using 1 workers.

Making splits..


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/mnist/incomplete.RCY2DQ_3.0.1/mnist-train.tfrecord*...:   0%|          | 0…

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/mnist/incomplete.RCY2DQ_3.0.1/mnist-test.tfrecord*...:   0%|          | 0/…

Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.
938
torch.Size([60000, 784])
100
torch.Size([10000, 784])
Train 50048
Val 10048
Test 10000


In [10]:
# Manifesting to memory of the device (we can afford it on colab ?)
train_batches = [[el.to(DEVICE) for el in batch] for batch in train_loader]
val_batches   = [[el.to(DEVICE) for el in batch] for batch in val_loader]
test_batches  = [[el.to(DEVICE) for el in batch] for batch in test_loader]

# Wrap for shuffling behavior
train_loader = ShuffledLoader(
    train_batches,
    shuffle_batches=True,
    shuffle_within_batch=True,
    device_for_randperm=DEVICE)

val_loader = val_batches
test_loader = test_batches

## **Configuration**

In [11]:
base_path = "/content/drive/MyDrive/HVAE/Table2_full_dataset/"
os.makedirs(base_path, exist_ok=True)

In [12]:
EPOCHS = 500
INPUT_DIM = 784
HIDDEN_DIM = 128

LATENT_MODE = 'sample'
PATIENCE = 50
WARMUP = 100
ONE_LAYER = False
LR = 1e-3

N_RUNS = 5

In [13]:
def run_and_test(mode, latent, test_dataloader, N):
    addon = "[SVAE]" if mode == "svae" else "[NVAE]"

    print(f"\n{addon} Instantiating SVAE and optimizer..")
    model = M1(mode,
                INPUT_DIM,
                HIDDEN_DIM,
                latent,
                ONE_LAYER,
                N_fit_clf=N
                )
    # To device
    model.to(DEVICE)

    optimizer = torch.optim.Adam(model.parameters(), lr=LR)

    print(f"{addon} Started training..")
    model, losses, all_parts = training_M1(train_loader,
                                    val_loader,
                                    model,
                                    optimizer,
                                    epochs=EPOCHS,
                                    beta_kl=1,
                                    warmup=WARMUP,
                                    patience=PATIENCE,
                                    show_loss_every=1,
                                    mode=LATENT_MODE)

    print(f"\n{addon} Predicting classes..")
    Y, Y_hat = predict_classes_loader(model, test_dataloader, LATENT_MODE)

    test_acc = accuracy_score(Y, Y_hat)
    print(f"{addon} Test accuracy: {test_acc*100:.2f}")
    return test_acc

In [14]:
def launch_experiment(N, test_dataloader):
    results_df = {"Latent Size":[], "Model":[], "Accuracy":[], "Std":[], "N_test":[]}
    for latent in tqdm([2, 5, 10, 20, 40], desc="Exploring latent.."):
        for mode in ["svae", "normal"]:
            accuracy_list = []
            for i in tqdm(list(range(N_RUNS)), desc="Repeated runs.."):
                print(f"\nSTARTING RUN n°{i+1}")
                accuracy_list.append(run_and_test(mode, latent, test_dataloader, N))
            avg_acc = np.nanmean(accuracy_list)
            std_acc = np.nanstd(accuracy_list)

            results_df["Latent Size"].append(latent)
            results_df["Model"].append(mode)
            results_df["Accuracy"].append(avg_acc)
            results_df["Std"].append(std_acc)
            results_df["N_test"].append(N)
    return pd.DataFrame(results_df)

## **$N = 100$**
Test Batch size is 100 so only test on 1 batch

In [ ]:
results_df_100 = launch_experiment(100, test_loader)

Exploring latent..:   0%|          | 0/5 [00:00<?, ?it/s]

Repeated runs..:   0%|          | 0/5 [00:00<?, ?it/s]


STARTING RUN n°1

[SVAE] Instantiating SVAE and optimizer..
[SVAE] Started training..


Epochs..:   0%|          | 0/500 [00:00<?, ?it/s]

Loss printing format:
epoch x: val = loss (-recon + beta_kl * kl) | train = loss (-recon + beta_kl * kl)

epoch 1: val = 179.3098 (179.3098 + 0.00 * 2.6683) | train = 195.6271 (195.6270 + 0.00 * 2.3031)
epoch 2: val = 170.9615 (170.9344 + 0.01 * 2.6862) | train = 173.9073 (173.8802 + 0.01 * 2.6863)
epoch 3: val = 167.5441 (167.4881 + 0.02 * 2.7676) | train = 168.5099 (168.4545 + 0.02 * 2.7488)
epoch 4: val = 163.0537 (162.9681 + 0.03 * 2.8293) | train = 165.0602 (164.9753 + 0.03 * 2.7990)
epoch 5: val = 161.6891 (161.5729 + 0.04 * 2.8764) | train = 162.4821 (162.3668 + 0.04 * 2.8545)
epoch 6: val = 159.8045 (159.6586 + 0.05 * 2.8890) | train = 160.6945 (160.5487 + 0.05 * 2.8853)
epoch 7: val = 159.7534 (159.5748 + 0.06 * 2.9474) | train = 160.5250 (160.3474 + 0.06 * 2.9303)
epoch 8: val = 158.3941 (158.1869 + 0.07 * 2.9290) | train = 158.8695 (158.6617 + 0.07 * 2.9381)
epoch 9: val = 157.9051 (157.6670 + 0.08 * 2.9463) | train = 157.9634 (157.7266 + 0.08 * 2.9320)
epoch 10: val = 156.7

...


STARTING RUN n°1

[SVAE] Instantiating SVAE and optimizer..
[SVAE] Started training..


Epochs..:   0%|          | 0/500 [00:00<?, ?it/s]

Loss printing format:
epoch x: val = loss (-recon + beta_kl * kl) | train = loss (-recon + beta_kl * kl)

epoch 1: val = 140.7604 (140.7604 + 0.00 * 9.8337) | train = 174.6625 (174.6625 + 0.00 * 8.1920)
epoch 2: val = 129.2164 (129.1158 + 0.01 * 9.9607) | train = 133.9524 (133.8522 + 0.01 * 9.9162)
epoch 3: val = 125.0510 (124.8493 + 0.02 * 9.9888) | train = 127.3920 (127.1904 + 0.02 * 9.9761)
epoch 4: val = 122.5017 (122.1987 + 0.03 * 9.9982) | train = 124.0986 (123.7958 + 0.03 * 9.9938)
epoch 5: val = 120.7344 (120.3302 + 0.04 * 10.0055) | train = 121.9855 (121.5812 + 0.04 * 10.0042)
epoch 6: val = 119.7813 (119.2754 + 0.05 * 10.0157) | train = 120.4946 (119.9890 + 0.05 * 10.0106)
epoch 7: val = 119.0618 (118.4547 + 0.06 * 10.0170) | train = 119.3207 (118.7137 + 0.06 * 10.0153)
epoch 8: val = 118.2043 (117.4959 + 0.07 * 10.0186) | train = 118.3920 (117.6836 + 0.07 * 10.0174)
epoch 9: val = 117.7823 (116.9726 + 0.08 * 10.0199) | train = 117.5857 (116.7761 + 0.08 * 10.0200)
epoch 10: v

Epochs..:   0%|          | 0/500 [00:00<?, ?it/s]

Loss printing format:
epoch x: val = loss (-recon + beta_kl * kl) | train = loss (-recon + beta_kl * kl)

epoch 1: val = 141.5466 (141.5465 + 0.00 * 9.7611) | train = 174.5374 (174.5374 + 0.00 * 7.9286)
epoch 2: val = 128.5298 (128.4294 + 0.01 * 9.9416) | train = 133.8302 (133.7305 + 0.01 * 9.8747)
epoch 3: val = 124.5200 (124.3184 + 0.02 * 9.9795) | train = 126.9032 (126.7019 + 0.02 * 9.9630)
epoch 4: val = 122.4159 (122.1130 + 0.03 * 9.9961) | train = 123.6295 (123.3268 + 0.03 * 9.9887)
epoch 5: val = 120.5716 (120.1675 + 0.04 * 10.0019) | train = 121.5008 (121.0967 + 0.04 * 10.0007)
epoch 6: val = 119.8151 (119.3096 + 0.05 * 10.0100) | train = 120.0017 (119.4963 + 0.05 * 10.0075)
epoch 7: val = 118.6340 (118.0270 + 0.06 * 10.0151) | train = 118.8248 (118.2182 + 0.06 * 10.0117)
epoch 8: val = 117.9670 (117.2586 + 0.07 * 10.0193) | train = 117.9273 (117.2190 + 0.07 * 10.0169)
epoch 9: val = 117.4869 (116.6771 + 0.08 * 10.0202) | train = 117.2037 (116.3941 + 0.08 * 10.0187)
epoch 10: v

In [ ]:
results_df_100.to_csv(base_path + "100_M1_results.csv")
results_df_100

## **$N = 600$**

In [ ]:
results_df_600 = launch_experiment(600, test_loader)

In [ ]:
results_df_600.to_csv(base_path + "600_M1_results.csv")
results_df_600

## **$N = 1000$**

In [ ]:
results_df_1000 = launch_experiment(1000, test_loader)

In [ ]:
results_df_1000.to_csv(base_path + "1000_M1_results.csv")
results_df_1000